# Naive Bayes

In [47]:
import csv
import random
import math

In [48]:
def loadcsv(filename):
    lines = csv.reader(open(filename, "r"));
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]

    return dataset

def splitdataset(dataset, splitratio):
    trainsize = int(len(dataset) * splitratio);
    trainset = []
    copy = list(dataset)

    while len(trainset) < trainsize:
        index = random.randrange(len(copy));
        trainset.append(copy.pop(index))
        
    return [trainset, copy]

def separatebyclass(dataset):
    separated = {}

    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
        
    return separated

In [49]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [50]:
def summarize(dataset): 
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)];
    del summaries[-1] 
    return summaries

def summarizebyclass(dataset):
    separated = separatebyclass(dataset);

    summaries = {}
    for classvalue, instances in separated.items():
        summaries[classvalue] = summarize(instances)

    return summaries   

In [51]:
def calculateprobability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateclassprobabilities(summaries, inputvector):
    probabilities = {}
    for classvalue, classsummaries in summaries.items():
        probabilities[classvalue] = 1
        for i in range(len(classsummaries)):
            mean, stdev = classsummaries[i] 
            x = inputvector[i] 
            probabilities[classvalue] *= calculateprobability(x, mean, stdev)
        return probabilities

In [52]:
def predict(summaries, inputvector): 
    probabilities = calculateclassprobabilities(summaries, inputvector)
    bestLabel, bestProb = None, -1

    for classvalue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classvalue

    return bestLabel

def getpredictions(summaries, testset):
    predictions = []

    for i in range(len(testset)):
        result = predict(summaries, testset[i])
        predictions.append(result)
        
    return predictions

In [53]:
def getaccuracy(testset, predictions):
    correct = 0
    for i in range(len(testset)):
        if testset[i][-1] == predictions[i]:
            correct += 1

    return (correct/float(len(testset))) * 100.0

## Main

In [54]:
splitratio = 0.7
dataset = loadcsv("../input/naivedataset/naivedata.csv");

trainingset, testset = splitdataset(dataset, splitratio)
print(f'Split {len(dataset)} rows into train = {len(trainingset)} and test = {len(testset)} rows')

summaries = summarizebyclass(trainingset)

predictions = getpredictions(summaries, testset)

accuracy = getaccuracy(testset, predictions)
print(f'Accuracy of the classifier is : {accuracy}%')

Split 768 rows into train = 537 and test = 231 rows
Accuracy of the classifier is : 67.53246753246754%
